"""

# Emotion Classification on Audio Data with PCA and AUC-ROC Evaluation



This notebook demonstrates an end-to-end pipeline for classifying emotions from audio data.

We use the following steps:



1. Load and label the dataset.

2. Extract MFCC features from audio files.

3. Apply PCA for dimensionality reduction.

4. Build an LSTM model for classification.

5. Evaluate the model with AUC-ROC and save the model.

6. Run the model on new .wav files and visualize results using ROC curves.



---



"""

In [1]:
!pip install pandas numpy seaborn matplotlib librosa keras tensorflow-macos --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 11.6 MB/s eta 0:00:00a 0:00:01


In [36]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout
from keras.metrics import AUC
import librosa
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')


In [19]:
# Path to dataset
PATH = '../wav/'

# Emotion codes and their meanings (from German emotion codes)
emotion_codes = {
    'W': 'anger',
    'L': 'boredom',
    'E': 'disgust',
    'A': 'anxiety',
    'F': 'happiness',
    'T': 'sadness',
    'N': 'neutral'
}

# Variables to hold data
file_emotion = []
file_path = []
speakers = []

# List of audio files in the directory
directory_list = os.listdir(PATH)

# metadata extraction including SpeakerID and Emotion
for file in directory_list:
    file_path.append(PATH + file)
    speakers.append(int(file[:2]))  # Speaker ID is the first two characters
    emotion_code = file[5]  # Emotion code is the sixth character
    file_emotion.append(emotion_codes.get(emotion_code, 'Unknown'))  # Map emotion code to emotion



# Create DataFrame with metadata
emodb_df = pd.DataFrame(list(zip(speakers, file_emotion, file_path)),
                        columns=['SpeakerID', 'Emotion', 'Path'])

emodb_df.head()

,SpeakerID,Emotion,Path
0,16,boredom,../wav/16a02Lb.wav
1,14,anger,../wav/14a07Wc.wav
2,10,anxiety,../wav/10a07Ad.wav
3,13,disgust,../wav/13a05Ea.wav
4,14,anger,../wav/14a05Wa.wav


In [24]:
emodb_df['Emotion'].value_counts()

Emotion
anger        127
boredom       81
neutral       79
happiness     71
anxiety       69
sadness       62
disgust       46
Name: count, dtype: int64

<Axes: xlabel='Emotion', ylabel='count'>

In [25]:
sns.countplot(data=emodb_df, x='Emotion')

In [27]:
# Feature extraction function with MFCC
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return np.mean(mfcc.T, axis=0)

# Extract MFCC features
X_mfcc = np.array([extract_mfcc(path) for path in tqdm(emodb_df['Path'], desc="Extracting MFCCs")])
X_mfcc.shape


Extracting MFCCs: 100%|██████████| 535/535 [00:05<00:00, 103.23it/s]


(535, 40)

In [28]:
# Apply PCA for dimensionality reduction
pca = PCA(n_components=20)  # Reducing to 20 components for efficiency
X_pca = pca.fit_transform(X_mfcc)



# Reshape data for LSTM input
X = np.expand_dims(X_pca, -1)
X.shape


(535, 20, 1)

In [29]:
# One-hot encode labels
enc = OneHotEncoder()
y = enc.fit_transform(emodb_df[['Emotion']]).toarray()
y.shape


(535, 7)

In [30]:
model = Sequential([
    LSTM(128, input_shape=(X.shape[1], 1), return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(emotion_codes), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[AUC(name="auc")])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,271 (294.03 KB)

 Trainable params: 75,271 (294.03 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Training the model
history = model.fit(X, y, validation_split=0.2, epochs=50, batch_size=64)


Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - auc: 0.5708 - loss: 1.9300 - val_auc: 0.5124 - val_loss: 1.9472
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.6499 - loss: 1.8477 - val_auc: 0.5180 - val_loss: 2.0142
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.6622 - loss: 1.8167 - val_auc: 0.5487 - val_loss: 1.9717
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.6879 - loss: 1.7995 - val_auc: 0.5729 - val_loss: 1.9384
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.7021 - loss: 1.7555 - val_auc: 0.6000 - val_loss: 1.9651
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.7371 - loss: 1.6737 - val_auc: 0.6520 - val_loss: 1.8758
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - auc: 0.7932 - loss: 1.5262 - val_auc: 0.7142 - val_loss: 1.7951
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.8090 - loss: 1.4843 - val_auc: 0.7330 - val_loss: 1.7117
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - auc: 0.8187 - loss: 1.4443 - 

In [32]:
# Predict probabilities for test data and calculate AUC-ROC

y_pred_proba = model.predict(X)
y_true = y
auc_roc = roc_auc_score(y_true, y_pred_proba, average="macro", multi_class="ovr")

print(f"AUC-ROC Score (Macro-Averaged): {auc_roc}")



# Optional: Classification Report
y_pred = np.argmax(y_pred_proba, axis=1)
y_true_classes = np.argmax(y_true, axis=1)
report = classification_report(y_true_classes, y_pred, target_names=enc.categories_[0])
print("Classification Report:")
print(report)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
AUC-ROC Score (Macro-Averaged): 0.9286713811497712
Classification Report:
              precision    recall  f1-score   support

       anger       0.70      0.83      0.76       127
     anxiety       0.78      0.65      0.71        69
     boredom       0.62      0.56      0.59        81
     disgust       0.52      0.65      0.58        46
   happiness       0.65      0.45      0.53        71
     neutral       0.64      0.73      0.69        79
     sadness       0.79      0.73      0.76        62

    accuracy                           0.67       535
   macro avg       0.67      0.66      0.66       535
weighted avg       0.68      0.67      0.67       535



In [33]:
model.save('AUC-ROC.h5')

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [38]:
# Predict probabilities and plot ROC curves for each emotion class
y_pred_proba = model.predict(X)

plt.figure(figsize=(10, 8))
for i, emotion in enumerate(enc.categories_[0]):
    fpr, tpr, _ = roc_curve(y[:, i], y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{emotion} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line for random guessing
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves by Emotion Class")
plt.legend(loc="lower right")
plt.show()


In [47]:
# Load the saved model
model = load_model("AUC-ROC.h5")

# Function to extract features from new .wav files and predict emotions
def predict_emotion_on_new_file(filepath):
    # Extract MFCC and apply PCA
    mfcc_features = extract_mfcc(filepath).reshape(1, -1)
    pca_features = pca.transform(mfcc_features)
    pca_features = np.expand_dims(pca_features, -1)

    # Predict probabilities
    predictions = model.predict(pca_features)
    predicted_emotion = enc.categories_[0][np.argmax(predictions)]
    return predicted_emotion, predictions

new_files = ['/Users/omedbahtuchi/Desktop/Masterarbeit/R_2dUWV4nZrCVjZBv_WOSECO.wav']
for file in new_files:
    emotion, proba = predict_emotion_on_new_file(file)
    print(f"File: {file}")
    print(f"Predicted Emotion: {emotion}")
    print("Probabilities:")
    for emotion, probability in zip(enc.categories_[0], proba[0]):
        print(f" {emotion}: {probability}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
File: /Users/omedbahtuchi/Desktop/Masterarbeit/R_2dUWV4nZrCVjZBv_WOSECO.wav
Predicted Emotion: neutral
Probabilities:
 anger: 0.29495251178741455
 anxiety: 0.0582905039191246
 boredom: 0.14871813356876373
 disgust: 0.006299983710050583
 happiness: 0.014791782014071941
 neutral: 0.47605663537979126
 sadness: 0.0008905039285309613


In [48]:
def predict_emotions_over_time(filepath, segment_duration=5.0, overlap=1.0):
    # Load audio file
    y, sr = librosa.load(filepath)
    
    # Calculate number of samples per segment
    segment_samples = int(segment_duration * sr)
    overlap_samples = int(overlap * sr)
    
    # Initialize an empty list to store results
    results = []
    
    # Process each segment
    for start in range(0, len(y) - segment_samples + 1, segment_samples - overlap_samples):
        # Extract segment
        segment = y[start:start + segment_samples]
        
        # Extract MFCC and apply PCA
        mfcc_features = np.mean(librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=40).T, axis=0).reshape(1, -1)
        pca_features = pca.transform(mfcc_features)
        pca_features = np.expand_dims(pca_features, -1)
        
        # Predict emotion probabilities
        predictions = model.predict(pca_features)
        predicted_emotion = enc.categories_[0][np.argmax(predictions)]
        
        # Calculate the timestamp for the segment
        start_time = start / sr
        end_time = (start + segment_samples) / sr
        
        # Append results
        results.append({
            "start_time": start_time,
            "end_time": end_time,
            "predicted_emotion": predicted_emotion,
            "probabilities": predictions[0]
        })

    return results

# Example usage
results = predict_emotions_over_time('/Users/omedbahtuchi/Desktop/Masterarbeit/R_2dUWV4nZrCVjZBv_WOSECO.wav')
for res in results:
    print(f"From {res['start_time']:.2f}s to {res['end_time']:.2f}s: {res['predicted_emotion']} (Probabilities: {res['probabilities']})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━